In [ ]:
import os
import os.path as osp
import subprocess
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from osgeo import gdal
import requests
import rasterio

# HRRR Rain Data Test

The purpose of this notebook is to investigate precipitation data from HRRR that are all zero. The bands we use are "PRATE", the precipitation rate, and "APCP", total accumulated precipitation. HRRR band numbers [definitions](https://www.nco.ncep.noaa.gov/pmb/products/hrrr/):
* 3D Pressure Levels ([source](https://www.nco.ncep.noaa.gov/pmb/products/hrrr/hrrr.t00z.wrfprsf00.grib2.shtml)):
    * `PRATE`: 628
    * `APCP`: 629
* Native Levels ([source](https://www.nco.ncep.noaa.gov/pmb/products/hrrr/hrrr.t00z.wrfnatf00.grib2.shtml)):
    * `PRATE`: 1053
    * `APCP`: 1054
* 2D Surface Levels ([source](https://www.nco.ncep.noaa.gov/pmb/products/hrrr/hrrr.t00z.wrfsfcf00.grib2.shtml)):
    * `PRATE`: 083
    * `APCP`: 084
 
Server of grib files linked from NOAA to: https://nomads.ncep.noaa.gov/pub/data/nccf/com/hrrr/prod/

    - *NOTE:* this url provides an html interface for current and previous day. The urls to the grib files for older days should still work for data, but the file directories cannot be viewed in a browser

## View HRRR Rain for Conus at different F Times

In [ ]:
ft = ["f00", "f01", "f02", "f03", "f04", "f05", "f06"]

In [ ]:
turl = "https://demo.openwfm.org/web/data/fmda/tif/20240420/hrrr.t00z.wrfprsf06.629.tif"
with rasterio.open(turl) as src: 
    plt.imshow(src.read(1))
    plt.title("test")

## Read FMDA Dict

In [ ]:
dict1 = pd.read_pickle("test_nw_202401_f03.pkl")

In [ ]:
# Print Forecast Hours in Data for 1st key

print(dict1[[*dict1.keys()][0]]["HRRR"].keys())

## Examine RAWS vs HRRR

We compare rain arrays for similarity. For simplicity, we filter out raws with missing data to avoid interpolation, which is done elsewhere in the workflow. 

In [ ]:
raws = []
hrrr1 = []
hrrr2 = []
hrrr3 = []

for k in dict1:
    if "rain" in dict1[k]["RAWS"].keys():
        # Only add when length is same
        if len(dict1[k]["RAWS"]["rain"][1:]) == len(dict1[k]["HRRR"]["f01"]["rain"][1:]):
            raws.append(dict1[k]["RAWS"]["rain"][1:])
            hrrr1.append(dict1[k]["HRRR"]["f01"]["rain"][1:])
            hrrr2.append(dict1[k]["HRRR"]["f02"]["rain"][1:])
            hrrr3.append(dict1[k]["HRRR"]["f03"]["rain"][1:])
        
raws = np.concatenate(raws)
hrrr1 = np.concatenate(hrrr1)
hrrr2 = np.concatenate(hrrr2)
hrrr3 = np.concatenate(hrrr3)

### Reconcile Units

* [HRRR](https://www.nco.ncep.noaa.gov/pmb/products/hrrr/hrrr.t00z.wrfprsf00.grib2.shtml): units for precipitation rate is $kg\cdot m^{-2}s^{-1}$
* RAWS units for precipitation is $mm\cdot h^{-1}$

We HRRR to $mm\cdot h^{-1}$. We utilize the fact that water has a specific density of roughly 1g per cubic cm. So 1km of water, or 1000g of water, spread over 1 $m^2$, or 10,000 $cm^2$, would cover that square meter to a depth of .1 cm, or 1mm. So 1 $kg\cdot m^{-2}s^{-1}$ is equal to 1 $mm\cdot s^{-1}$, so we just need to multiply by 3600 seconds per hour to get units of $mm\cdot h^{-1}$.

In [ ]:
# Confirm units from RAWS data
from synoptic.services import variables
variables(verbose="HIDE").loc["precip_accum"]

In [ ]:
hrrr1 = hrrr1*3600
hrrr2 = hrrr2*3600
hrrr3 = hrrr3*3600

Plot Relationship

In [ ]:
plt.plot(raws, hrrr1, 'o', label = "f01")
plt.plot(raws, hrrr2, 'o', label = "f02")
plt.plot(raws, hrrr3, 'o', label = "f03")
plt.legend()

### Summarize raw correlation

In [ ]:
print("Correlations between RAWS and HRRR rain")
print(f"F01: {np.corrcoef(raws, hrrr1)}")
print(f"F02: {np.corrcoef(raws, hrrr2)}")
print(f"F03: {np.corrcoef(raws, hrrr3)}")